In [1]:
from process_wikipedia import *

WIKIPEDIA_ROOT = "/home/jeff/data/wikipedia2"
WIKIPEDIA_DL = os.path.join(WIKIPEDIA_ROOT, 'dl')

# Download Wikipedia Dump Data

This part only needs to be run once, at the beginning to download Wikipedia to somewhere on your system.  Note, this part will take a long time!  It will transfer much data!

In [2]:
wikidump_download(WIKIPEDIA_DL)

https://dumps.wikimedia.org/enwiki/20210320


NameError: name 'bs4' is not defined

# Process Wikipedia Data

This can be run after the previous section has downloaded Wikipedia data.  This example creates 3 files from extracted Wikipedia data:

* article.csv - A listing of all articles on Wikipedia, with their Wikipedia ID.
* redirect.csv - A listing of all redirects of articles on Wikipedia. e.g. USA to United_States
* template.csv - A listing of all templates on Wikipedia.  These are the "types" of articles.

In [ ]:
class ProcessPagesWorker():
    def __init__(self, config, outputQueue):
        self.config = config
        self.outputQueue = outputQueue
        
    def process_template(self, id, title):
        self.outputQueue.put(
            {'template': [id, title] }
        )
    
    def process_article(self, id, title):
        self.outputQueue.put(
            {'article': [id, title] }
        )
    
    def process_redirect(self, id, title, redirect):
        self.outputQueue.put(
            {'redirect': [id, title, redirect] }
        )
        
    def report_progress(self, completed):
        self.outputQueue.put({"completed": completed})
        
    def close(self):
        self.articles_fp.close()
        self.redirect_fp.close()
        self.template_fp.close()

class ProcessPages:
    def __init__(self, output_path):        
        pathArticles = os.path.join(output_path, "article.csv")
        pathRedirect = os.path.join(output_path, "redirect.csv")
        pathTemplate = os.path.join(output_path, "template.csv")
        
        self.articles_fp = codecs.open(pathArticles, "w", ENCODING)
        self.redirect_fp = codecs.open(pathRedirect, "w", ENCODING)
        self.template_fp = codecs.open(pathTemplate, "w", ENCODING)
    
        self.articlesWriter = csv.writer(self.articles_fp, quoting=csv.QUOTE_MINIMAL)
        self.redirectWriter = csv.writer(self.redirect_fp, quoting=csv.QUOTE_MINIMAL)
        self.templateWriter = csv.writer(self.template_fp, quoting=csv.QUOTE_MINIMAL)
        
        self.articlesWriter.writerow(['id', 'title'])
        self.redirectWriter.writerow(['id', 'title', 'redirect'])
        self.templateWriter.writerow(['id', 'title'])
    
    def handle_event(self, evt):

        if "article" in evt:
            self.articlesWriter.writerow(evt['article'])
        elif "template" in evt:
            self.templateWriter.writerow(evt['template'])
        elif "redirect" in evt:
            self.redirectWriter.writerow(evt['redirect'])
            
    def get_worker_class(self, outputQueue, config):
        return ProcessPagesWorker(config, outputQueue)
    

wiki = ExtractWikipedia(
    ProcessPages(WIKIPEDIA_ROOT), # where you want the extracted Wikipedia files to go
    WIKIPEDIA_DL #Location you downloaded Wikipedia to
)
wiki.process()